In [1]:
import numpy as np
from numpy.linalg import norm
import pickle
import os
import time
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import requests
import PIL
from io import BytesIO

In [2]:
URL = "http://olidawiki.s3-website.eu-central-1.amazonaws.com/c676eba0-78ee-42f4-a48c-758cdb77a5f8.jpg"

model = ResNet50(weights='imagenet', include_top=False,
                 input_shape=(224, 224, 3), pooling='avg')

def extract_features(img_url, model):
    input_shape = (224, 224, 3)
    r = requests.get(img_url)
    img = PIL.Image.open(BytesIO(r.content)).resize((224, 224)).convert("RGB")
    
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

94765736/94765736 [==============================] - 4s 0us/step


In [3]:
features = extract_features("http://olidawiki.s3-website.eu-central-1.amazonaws.com/3295f28c-b5c7-4437-b6c0-79efc55f5c44.jpg", model)

1/1 [==============================] - 1s 826ms/step


In [4]:
features

array([0.00019904, 0.00011626, 0.00213826, ..., 0.00818219, 0.00533593,
       0.00735781], dtype=float32)

In [5]:
!pip install qdrant-client

     |████████████████████████████████| 90 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 5.9 MB/s  eta 0:00:01
     |████████████████████████████████| 2.4 MB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 3.2 MB 24.1 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 8.0 MB/s  eta 0:00:01
     |████████████████████████████████| 57 kB 4.7 MB/s  eta 0:00:01
     |████████████████████████████████| 409 kB 30.3 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 9.7 MB/s  eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the defaul

In [9]:
from qdrant_client import QdrantClient

client = QdrantClient(host="qdrant", port=6333)